In [1]:
import numpy as np

from numba import njit, uint64
from numba.typed import Dict
from hftbacktest.stats import LinearAssetRecord


from hftbacktest import (
    BacktestAsset,
    ROIVectorMarketDepthBacktest,
    GTX,
    LIMIT,
    BUY,
    SELL,
    BUY_EVENT,
    SELL_EVENT,
    Recorder
)
from hftbacktest.stats import LinearAssetRecord

@njit
def obi_mm(
    hbt,
    stat,
    half_spread,
    skew,
    c1,
    looking_depth,
    interval,
    window,
    order_qty_dollar,
    max_position_dollar,
    grid_num,
    grid_interval,
    roi_lb,
    roi_ub
):
    asset_no = 0
    imbalance_timeseries = np.full(30_000_000, np.nan, np.float64)

    tick_size = hbt.depth(0).tick_size
    lot_size = hbt.depth(0).lot_size

    t = 0
    roi_lb_tick = int(round(roi_lb / tick_size))
    roi_ub_tick = int(round(roi_ub / tick_size))

    while hbt.elapse(interval) == 0:
        hbt.clear_inactive_orders(asset_no)

        depth = hbt.depth(asset_no)
        position = hbt.position(asset_no)
        orders = hbt.orders(asset_no)

        best_bid = depth.best_bid
        best_ask = depth.best_ask

        mid_price = (best_bid + best_ask) / 2.0

        sum_ask_qty = 0.0
        from_tick = max(depth.best_ask_tick, roi_lb_tick)
        upto_tick = min(int(np.floor(mid_price * (1 + looking_depth) / tick_size)), roi_ub_tick)
        for price_tick in range(from_tick, upto_tick):
            sum_ask_qty += depth.ask_depth[price_tick - roi_lb_tick]

        sum_bid_qty = 0.0
        from_tick = min(depth.best_bid_tick, roi_ub_tick)
        upto_tick = max(int(np.ceil(mid_price * (1 - looking_depth) / tick_size)), roi_lb_tick)
        for price_tick in range(from_tick, upto_tick, -1):
            sum_bid_qty += depth.bid_depth[price_tick - roi_lb_tick]

        imbalance_timeseries[t] = sum_bid_qty - sum_ask_qty

        # Standardizes the order book imbalance timeseries for a given window
        m = np.nanmean(imbalance_timeseries[max(0, t + 1 - window):t + 1])
        s = np.nanstd(imbalance_timeseries[max(0, t + 1 - window):t + 1])
        alpha = np.divide(imbalance_timeseries[t] - m, s)

        #--------------------------------------------------------
        # Computes bid price and ask price.

        order_qty = max(round((order_qty_dollar / mid_price) / lot_size) * lot_size, lot_size)
        fair_price = mid_price + c1 * alpha

        normalized_position = position / order_qty

        reservation_price = fair_price - skew * normalized_position

        bid_price = min(np.round(reservation_price - half_spread), best_bid)
        ask_price = max(np.round(reservation_price + half_spread), best_ask)

        bid_price = np.floor(bid_price / tick_size) * tick_size
        ask_price = np.ceil(ask_price / tick_size) * tick_size

        #--------------------------------------------------------
        # Updates quotes.

        # Creates a new grid for buy orders.
        new_bid_orders = Dict.empty(np.uint64, np.float64)
        if position * mid_price < max_position_dollar and np.isfinite(bid_price):
            for i in range(grid_num):
                bid_price_tick = round(bid_price / tick_size)

                # order price in tick is used as order id.
                new_bid_orders[uint64(bid_price_tick)] = bid_price

                bid_price -= grid_interval

        # Creates a new grid for sell orders.
        new_ask_orders = Dict.empty(np.uint64, np.float64)
        if position * mid_price > -max_position_dollar and np.isfinite(ask_price):
            for i in range(grid_num):
                ask_price_tick = round(ask_price / tick_size)

                # order price in tick is used as order id.
                new_ask_orders[uint64(ask_price_tick)] = ask_price

                ask_price += grid_interval

        order_values = orders.values();
        while order_values.has_next():
            order = order_values.get()
            # Cancels if a working order is not in the new grid.
            if order.cancellable:
                if (
                    (order.side == BUY and order.order_id not in new_bid_orders)
                    or (order.side == SELL and order.order_id not in new_ask_orders)
                ):
                    hbt.cancel(asset_no, order.order_id, False)

        for order_id, order_price in new_bid_orders.items():
            # Posts a new buy order if there is no working order at the price on the new grid.
            if order_id not in orders:
                hbt.submit_buy_order(asset_no, order_id, order_price, order_qty, GTX, LIMIT, False)

        for order_id, order_price in new_ask_orders.items():
            # Posts a new sell order if there is no working order at the price on the new grid.
            if order_id not in orders:
                hbt.submit_sell_order(asset_no, order_id, order_price, order_qty, GTX, LIMIT, False)

        t += 1

        if t >= len(imbalance_timeseries):
            raise Exception

        # Records the current state for stat calculation.
        stat.record(hbt)

In [2]:

data = np.concatenate(
[np.load('data/binance_spot/solfdusd_{}.npz'.format(date))['data'] for date in [20251011, 20251012]]
)
initial_snapshot = np.load('data/binance_spot/solfdusd_20251010_eod.npz')['data']
latency_data = np.concatenate(
[np.load('data/binance_spot/solfdusd_{}_latency.npz'.format(date))['data'] for date in [20251011, 20251012]]
)


In [3]:
def test(p):
    roi_lb = 50
    roi_ub = 500

    asset = (
        BacktestAsset()
            .data(data)
            .initial_snapshot(initial_snapshot)
            .linear_asset(1.0)
            .intp_order_latency(latency_data)
            .power_prob_queue_model(2)
            .no_partial_fill_exchange()
            .trading_value_fee_model(0.0000, 0.0003)
            .tick_size(0.01)
            .lot_size(0.001)
            .roi_lb(roi_lb)
            .roi_ub(roi_ub)
    )

    hbt = ROIVectorMarketDepthBacktest([asset])

    recorder = Recorder(1, 30_000_000)

    # half_spread = half_spread
    # skew = skew
    # c1 =  c1
    # depth =  depth  # 2.5% from the mid price
    interval = 1_000_000_000 # 1s
    window = 3_600_000_000_000 / interval # 1hour
    order_qty_dollar = 50
    max_position_dollar = order_qty_dollar * 50
    grid_num = 1
    grid_interval = hbt.depth(0).tick_size

    obi_mm(
        hbt,
        recorder.recorder,
        p['half_spread'],
        p['skew'],
        p['c1'],
        p['depth'],
        interval,
        window,
        order_qty_dollar,
        max_position_dollar,
        grid_num,
        grid_interval,
        roi_lb,
        roi_ub
    )

    hbt.close()

    stats = LinearAssetRecord(recorder.get(0)).stats(book_size=10_000)
    return stats.splits[0]['Return']


In [ ]:
# pip install optuna numpy pandas
import optuna, numpy as np, pandas as pd
from optuna.trial import TrialState

TICK_SIZE = 0.01

def objective(trial: optuna.Trial):
    # 搜索空间（用 tick 搜，落地时乘以 TICK_SIZE）
    hs_t = trial.suggest_int("half_spread_ticks", 10, 2000)
    sk_t = trial.suggest_int("skew_ticks",        2,  1000)
    c1_t = trial.suggest_int("c1_ticks",         10, 1000)
    dp   = trial.suggest_int("depth", 1, 1000, step=1) #万分之

    params = dict(
        half_spread = hs_t * TICK_SIZE,
        skew        = sk_t * TICK_SIZE,
        c1          = c1_t * TICK_SIZE,
        depth       = dp * 1e4
    )
    # 把“已换算后的真实参数”存进 user_attrs，便于后面导出
    trial.set_user_attr("half_spread", params["half_spread"])
    trial.set_user_attr("skew",        params["skew"])
    trial.set_user_attr("c1",          params["c1"])
    trial.set_user_attr("depth",       params["depth"])

    returns = test(params)
    return returns 

# 优化
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=3 * 1e3, n_jobs=1)  

# ===== 取出 Top-100 并导出 =====
trials = [t for t in study.get_trials(states=(TrialState.COMPLETE,)) if t.value is not None]
trials_sorted = sorted(trials, key=lambda t: t.value, reverse=True)  # 因为是 maximize
top100 = trials_sorted[:100]

rows = []
for i, t in enumerate(top100, 1):
    row = {"rank": i, "trial": t.number, "score": t.value}
    # 原始搜索维度（tick）
    row.update({k: v for k, v in t.params.items()})
    # 方便阅读的“换算后参数”（价格单位 & 比例）
    row.update(t.user_attrs)
    rows.append(row)

df = pd.DataFrame(rows)
print(df.head(10))                  # 预览
df.to_csv("optuna_top100.csv", index=False)  # 导出 Top-100


/opt/homebrew/Caskroom/miniconda/base/envs/hfbacktest/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-10-13 15:52:14,989] A new study created in memory with name: no-name-743bfc03-b023-4aac-8ac5-0ce2d146d235
[I 2025-10-13 15:52:28,048] Trial 0 finished with value: -0.001757587000000006 and parameters: {'half_spread_ticks': 755, 'skew_ticks': 951, 'c1_ticks': 735, 'depth': 599}. Best is trial 0 with value: -0.001757587000000006.
[I 2025-10-13 15:52:37,895] Trial 1 finished with value: 2.8309999999999697e-05 and parameters: {'half_spread_ticks': 320, 'skew_ticks': 157, 'c1_ticks': 67, 'depth': 867}. Best is trial 1 with value: 2.8309999999999697e-05.
/opt/homebrew/Caskroom/miniconda/base/envs/hfbacktest/lib/python3.10/site-packages/hftbacktest/stats/metrics.py:102: RuntimeWarning: invalid value en